## Example 2: Convert TFHub Models to OpenVINO IR

This notebook shows how to convert TFHub models to OpenVINO IR format.

See list of supported models in TFHub-SupportedModelList

Following are the steps
- Install necessary pip packages.
- In AWS Sagemaker environment, remove few unused conda envs. This will ensure we have enought space for OpenVINO docker images.
- Select a model from the supported list
- Run the conversion

### Install necessary pip packages

In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

### Removed few unused conda envs. 

In [ ]:
!bash remove-unused-conda-envs.sh

## Convert TFHub Models to OpenVINO IR

In [ ]:
from ov_utils import create_ir
import csv

### Provide below details:

`bucket_name` - S3 bucket name with "sagemaker" as part of the name.
Example: sagemaker-ir-creation

`tfhub_model_url` - TfHub model URL. See supported list in TFHub-SupportedModelList

`input_shape` - Input shape of the selected model. See TFHub-SupportedModelList

`precision_type` - Use either FP32 or FP16

#### NOTE: If the bucket does not exist then it will be created.

#### Example:
```
bucket_name = "sagemaker-ir-creation"
tfhub_model_url = "https://tfhub.dev/google/efficientnet/b0/classification/1"
input_shape = "1,224,224,3"
precision_type = "FP32"
```

In [ ]:
bucket_name = "sagemaker-ir-creation" # must have sagemaker as part of the name
tfhub_model_url = "Pick-supported-model-url"
input_shape = "model-input-shape"
precision_type = "FP32" # FP16 also supported

#### Create IR params.

In [ ]:
url_arr = tfhub_model_url.split("/")
url_arr_len = len(url_arr)
model_name = "".join([url_arr[url_arr_len - 4], "-", url_arr[url_arr_len - 3]])
output_dir = "".join(["./", model_name.replace('.','-'), "-tfhub-", precision_type])
if isinstance(input_shape, str):
    input_shape = [int(i) for i in input_shape.split(",")]

create_ir_params = {
    "tfhub_model_url": tfhub_model_url,
    "output_dir": output_dir,
    "mo_params": {
        "input_shape": input_shape,
        "data_type": precision_type,
        "model_name": model_name
    },
    "bucket_name": bucket_name
}

### Start the IR creation and upload to S3.

In [ ]:
try:
    create_ir(create_ir_params)
    print (f"IR files created for model:{model_name} and the same is uploaded in S3:{bucket_name}")
except Exception as err:
    print(f"FAILED: Please find the error details below:")
    print(err)